In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
train_dir = r'dataset/train'
validation_dir = r'dataset/val'


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 1951 images belonging to 4 classes.
Found 324 images belonging to 4 classes.


In [6]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [7]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)


In [8]:
model = Model(inputs=base_model.input, outputs=x)

In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10


C:\Users\hakgs\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 635s 10s/step - accuracy: 0.4071 - loss: -31.2124 - val_accuracy: 0.3063 - val_loss: -162.0849
Epoch 2/10
 1/60 ━━━━━━━━━━━━━━━━━━━━ 7:40 8s/step - accuracy: 0.5625 - loss: -60.0406

C:\Users\hakgs\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.5625 - loss: -60.0406 - val_accuracy: 0.7500 - val_loss: -69.6312
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 598s 10s/step - accuracy: 0.4304 - loss: -162.7016 - val_accuracy: 0.3156 - val_loss: -375.6336
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.3125 - loss: -434.4304 - val_accuracy: 0.0000e+00 - val_loss: -801.9688
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 647s 11s/step - accuracy: 0.4095 - loss: -357.1199 - val_accuracy: 0.3156 - val_loss: -675.6562
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.4688 - loss: -283.7935 - val_accuracy: 0.0000e+00 - val_loss: -1175.2935
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 661s 11s/step - accuracy: 0.4037 - loss: -651.8691 - val_accuracy: 0.3094 - val_loss: -1061.7640
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.5000 - loss: -1137.4034 - val_accuracy: 0.5000 - val_loss: -45.5085
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 654s 11s/step - accuracy: 0.

In [12]:
model.save('VGGnet.h5')